In [2]:
import pandas as pd
import requests
import os
import time
from trafilatura import fetch_url, extract
from urllib.parse import urlparse, urljoin

# ====================================================================
# KONFIGURATION
# ====================================================================

INPUT_FILE = "WikiNasdaq_100_constituents.csv"
OUTPUT_FILE = "nasdaq100_company_texts_V2.csv"
MAX_TEXT_LENGTH = 3000
REQUEST_TIMEOUT = 10 
# WICHTIG: Verzögerung, um Rate Limiting zu vermeiden (mind. 1-2 Sekunden empfohlen!)
SCRAPING_DELAY = 1.5 

# Liste der relativen Pfade, die nacheinander versucht werden
TARGET_PATHS = ['/company', '/about', '/investors', '/']

# ====================================================================
# FUNKTIONEN
# ====================================================================

def is_valid_url(url):
    """Prüft, ob die URL gültig und ein http/https-Schema verwendet."""
    if pd.isna(url) or str(url).lower() in ['n/a', 'nan']:
        return False
    try:
        result = urlparse(url)
        return all([result.scheme in ['http', 'https'], result.netloc])
    except ValueError:
        return False

def build_url(base_url, path):
    """Baut eine vollständige URL aus der Basis-URL und dem relativen Pfad."""
    if not base_url.endswith('/') and not path.startswith('/'):
        return base_url + '/' + path
    return urljoin(base_url, path)

def scrape_content_with_retry(base_url):
    """
    Versucht nacheinander, Inhalte von definierten Zielpfaden zu scrapen.
    """
    for path in TARGET_PATHS:
        url_to_scrape = build_url(base_url, path)
        print(f"   -> Versuche Pfad: {url_to_scrape}")
        
        try:
            # Füge einen kleinen Delay vor jedem Request ein
            time.sleep(SCRAPING_DELAY) 
            
            # Trafilatura lädt die Seite herunter
            downloaded = fetch_url(url_to_scrape)
            
            if downloaded:
                # Extrahieren des reinen Textes
                extracted_json = extract(downloaded, output_format='json', 
                                         include_comments=False,
                                         include_images=False,
                                         include_tables=False)
                
                if extracted_json:
                    data = json.loads(extracted_json)
                    text = data.get('text', '')
                    
                    # Logik: Wenn mehr als 100 Zeichen extrahiert wurden, nehmen wir den Inhalt an
                    if len(text) > 100 and not text.startswith("FEHLER_BEI_ABRUF"):
                        print(f"   -> ✅ Erfolg auf Pfad: {path}")
                        return text
            
        except requests.exceptions.Timeout:
            print(f"   -> ⚠️ Timeout beim Abruf von {url_to_scrape}")
            return "FEHLER_BEI_ABRUF: Timeout"
        except Exception as e:
            # Ignoriere allgemeine Fehler und versuche den nächsten Pfad
            continue

    return "KEIN_HAUPTINHALT_GEFUNDEN_NACH_FALLBACK"


# ====================================================================
# HAUPTSKRIPT
# ====================================================================

import json

# 1. Daten laden
df = pd.read_csv(INPUT_FILE)
df['Extracted_Text'] = None  # Neue Spalte initialisieren

print(f"Starte gezieltes Web-Scraping für {len(df)} Unternehmen...")

# 2. Schleife über alle Unternehmen
for index, row in df.iterrows():
    ticker = row['Ticker']
    website = row['Website']
    
    if not is_valid_url(website):
        df.loc[index, 'Extracted_Text'] = "URL_UNGÜLTIG"
        print(f"[{ticker}] ❌ URL ungültig. Überspringe.")
        continue

    print(f"\n--- [{ticker}] Starte Abruf: {website} ---")
    
    extracted_content = scrape_content_with_retry(website)
    
    # 3. Ergebnis verarbeiten
    if extracted_content.startswith("FEHLER_BEI_ABRUF"):
        df.loc[index, 'Extracted_Text'] = extracted_content
    elif extracted_content == "KEIN_HAUPTINHALT_GEFUNDEN_NACH_FALLBACK":
         df.loc[index, 'Extracted_Text'] = "KEIN_HAUPTINHALT_GEFUNDEN"
    else:
        # Text auf die maximale Länge kürzen und speichern
        short_text = extracted_content[:MAX_TEXT_LENGTH]
        df.loc[index, 'Extracted_Text'] = short_text
        print(f"[{ticker}] ✅ Endgültiger Erfolg! {len(short_text)} Zeichen extrahiert.")

print("\n====================================")
# 4. Resultierendes DataFrame speichern
df.to_csv(OUTPUT_FILE, index=False)
print(f"Datenextraktion abgeschlossen. Resultate gespeichert in: {OUTPUT_FILE}")
print("====================================")

Starte gezieltes Web-Scraping für 102 Unternehmen...

--- [ADBE] Starte Abruf: https://www.adobe.com/ ---
   -> Versuche Pfad: https://www.adobe.com/company
   -> ✅ Erfolg auf Pfad: /company
[ADBE] ✅ Endgültiger Erfolg! 1927 Zeichen extrahiert.

--- [AMD] Starte Abruf: https://www.amd.com/ ---
   -> Versuche Pfad: https://www.amd.com/company
   -> Versuche Pfad: https://www.amd.com/about
   -> Versuche Pfad: https://www.amd.com/investors
   -> Versuche Pfad: https://www.amd.com/

--- [ABNB] Starte Abruf: https://www.airbnb.com/ ---
   -> Versuche Pfad: https://www.airbnb.com/company
   -> Versuche Pfad: https://www.airbnb.com/about
   -> ✅ Erfolg auf Pfad: /about
[ABNB] ✅ Endgültiger Erfolg! 3000 Zeichen extrahiert.
[GOOGL] ❌ URL ungültig. Überspringe.
[GOOG] ❌ URL ungültig. Überspringe.

--- [AMZN] Starte Abruf: https://www.amazon.com/ ---
   -> Versuche Pfad: https://www.amazon.com/company
   -> Versuche Pfad: https://www.amazon.com/about
   -> ✅ Erfolg auf Pfad: /about
[AMZN] ✅ Endg